# HW2


文本欄位：text <br>
目標類別欄位：label

目標類別有14個 <br>
評估指標：多元分類的 marco-F1

訓練資料檔：ptt_data_hw.csv

測試題目檔：ptt_data_test_63897.csv <br>
答案檔：ptt_data_submisstion_63897.csv <br>
以上兩個檔案將於12月12日晚上公布 <br>
請根據「測試題目檔」的文字預測類別，並將預測答案填入「答案檔」

In [2]:
import numpy as np
import pandas as pd
# import gdown
import jieba

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## download data

In [53]:
data = pd.read_csv("ptt_data_hw.csv")
data.fillna({'text':''}, inplace=True) #.dropna(inplace=True)
data

,Unnamed: 0,text,label
0,0,跟阿KEN不是沒有嗎?? 不過她生日願望是變紙片人耶!! 看來是,Beauty
1,1,150的話我收了....XD,car
2,2,推!!,car
3,3,哪一個夜市啊???,Lifeismoney
4,4,是讓渡條約唷 不錯唷~~~~推!!!,Baseball
...,...,...,...
129725,129725,非常好的一篇導讀想法(y),movie
129726,129726,3樓,NBA
129727,129727,我剛剛好像看到水果奶奶....,LoL
129728,129728,我騎迅光,Stock


In [3]:
data['LABEL'] = LabelEncoder().fit_transform(data['label'])
data['LABEL']

0          1
1         10
2         10
3          6
4          0
          ..
129725    12
129726     8
129727     7
129728     9
129729     7
Name: LABEL, Length: 129115, dtype: int32

In [4]:
data['title_SEG'] = [jieba.lcut(sent) for sent in data['text']]
data['title_SEG'] = data['title_SEG'].apply(lambda x:' '.join(x))
data.head(5)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\M07682\AppData\Local\Temp\jieba.cache
Loading model cost 0.828 seconds.
Prefix dict has been built successfully.


,Unnamed: 0,text,label,LABEL,title_SEG
0,0,跟阿KEN不是沒有嗎?? 不過她生日願望是變紙片人耶!! 看來是,Beauty,1,跟 阿 KEN 不是 沒有 嗎 ? ? 不過 她 生日 願望 是 變紙片 人 耶 ! !...
1,1,150的話我收了....XD,car,10,150 的 話 我 收 了 .... XD
2,2,推!!,car,10,推 ! !
3,3,哪一個夜市啊???,Lifeismoney,6,哪一個 夜市 啊 ? ? ?
4,4,是讓渡條約唷 不錯唷~~~~推!!!,Baseball,0,是 讓 渡條 約 唷 不錯 唷 ~ ~ ~ ~ 推 ! ! !


## Split dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data['title_SEG'], data['LABEL'], test_size=0.33, random_state=42)

In [6]:
print(y_train.shape)
print(y_test.shape)

(86507,)
(42608,)


In [7]:
def show_result(predicted, predicted_proba, target):
    print('*'*50)
    print('predicted class of first 3 samples')
    print(predicted[:3])

    print('*'*50)
    print('predicted class proba. of first 3 samples')
    print(predicted_proba[:3])


    accuracy = accuracy_score(target, predicted)
    print('Accuracy:', accuracy)

    report = classification_report(target, predicted)
    print(report)

## Feature vectorization

### Using CountVectorizer

In [8]:
# create feature vectors
count_vect = CountVectorizer(max_features=1000) # max_features=130107
X_train_counts = count_vect.fit_transform(X_train)

#prints the train data shape
print('train data shape using CountVectorizer')
print(X_train_counts.shape)

#prints the test data shape
X_test_counts = count_vect.transform(X_test)
print('test data shape using CountVectorizer')
print(X_test_counts.shape)

train data shape using CountVectorizer
(86507, 1000)
test data shape using CountVectorizer
(42608, 1000)


### Using TfidfVectorizer

In [9]:
# create feature vectors
tfidf_vect = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vect.fit_transform(X_train)

#prints the train data shape
print('train data shape using TfidfVectorizer')
print(X_train_tfidf.shape)

#prints the test data shape
X_test_tfidf = tfidf_vect.transform(X_test)
print('test data shape using TfidfVectorizer')
print(X_test_tfidf.shape)

train data shape using TfidfVectorizer
(86507, 1000)
test data shape using TfidfVectorizer
(42608, 1000)


## Create classifier

### Naive Bayes classifier with CountVectorizer

In [10]:
# Create classifier and use count vectors
MultinomialNB_clf = MultinomialNB()
print('*'*50)
print('MultinomialNB classifier with CountVectorizer')
print(MultinomialNB_clf)

# fit train data
MultinomialNB_clf.fit(X_train_counts, y_train)

# predict the class and class proba.
predicted = MultinomialNB_clf.predict(X_test_counts)
predicted_proba = MultinomialNB_clf.predict_proba(X_test_counts)

show_result(predicted, predicted_proba, y_test)

**************************************************
MultinomialNB classifier with CountVectorizer
MultinomialNB()
**************************************************
predicted class of first 3 samples
[7 7 8]
**************************************************
predicted class proba. of first 3 samples
[[0.08817784 0.07731166 0.07665276 0.08992336 0.09001584 0.02993977
  0.06242269 0.09985319 0.08666351 0.07697643 0.08608552 0.07004057
  0.06593686]
 [0.08817784 0.07731166 0.07665276 0.08992336 0.09001584 0.02993977
  0.06242269 0.09985319 0.08666351 0.07697643 0.08608552 0.07004057
  0.06593686]
 [0.03277783 0.0388381  0.02784911 0.05770882 0.12595691 0.08103591
  0.07893342 0.07284825 0.14855717 0.14436943 0.07198055 0.05050902
  0.06863548]]
Accuracy: 0.23993146826886969
              precision    recall  f1-score   support

           0       0.27      0.15      0.19      3816
           1       0.26      0.22      0.24      3301
           2       0.27      0.13      0.17      3286
  

### Naive Bayes classifier with TfidfVectorizer

In [11]:
# Create classifier and use tf-idf vectors
MultinomialNB_clf = MultinomialNB()
print('*'*50)
print('MultinomialNB classifier with TfidfVectorizer')
print(MultinomialNB_clf)

# fit train data
MultinomialNB_clf.fit(X_train_tfidf, y_train)

# predict the class and class proba.
predicted = MultinomialNB_clf.predict(X_test_tfidf)
predicted_proba = MultinomialNB_clf.predict_proba(X_test_tfidf)

show_result(predicted, predicted_proba, y_test)

**************************************************
MultinomialNB classifier with TfidfVectorizer
MultinomialNB()
**************************************************
predicted class of first 3 samples
[7 7 9]
**************************************************
predicted class proba. of first 3 samples
[[0.08817784 0.07731166 0.07665276 0.08992336 0.09001584 0.02993977
  0.06242269 0.09985319 0.08666351 0.07697643 0.08608552 0.07004057
  0.06593686]
 [0.08817784 0.07731166 0.07665276 0.08992336 0.09001584 0.02993977
  0.06242269 0.09985319 0.08666351 0.07697643 0.08608552 0.07004057
  0.06593686]
 [0.0388081  0.04521797 0.03035792 0.05988506 0.1179743  0.08084531
  0.0771825  0.08729007 0.12928463 0.13935126 0.07688681 0.0479118
  0.06900427]]
Accuracy: 0.2387579797221179
              precision    recall  f1-score   support

           0       0.27      0.15      0.20      3816
           1       0.27      0.22      0.24      3301
           2       0.27      0.12      0.17      3286
    

### KNN classifier with CountVectorizer

In [12]:
# Create classifier and use count vectors
KNeighborsClassifier_clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
print('*'*50)
print('KNeighbors classifier with CountVectorizer')
print(KNeighborsClassifier_clf)

# fit train data
KNeighborsClassifier_clf.fit(X_train_counts, y_train)

# predict the class and class proba.
predicted = KNeighborsClassifier_clf.predict(X_test_counts)
predicted_proba = KNeighborsClassifier_clf.predict_proba(X_test_counts)

show_result(predicted, predicted_proba, y_test)

**************************************************
KNeighbors classifier with CountVectorizer
KNeighborsClassifier(weights='distance')
**************************************************
predicted class of first 3 samples
[3 3 5]
**************************************************
predicted class proba. of first 3 samples
[[0.   0.   0.   0.2  0.   0.   0.   0.2  0.2  0.   0.   0.2  0.2 ]
 [0.   0.   0.   0.2  0.   0.   0.   0.2  0.2  0.   0.   0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.25 0.   0.25 0.   0.25 0.25 0.   0.  ]]
Accuracy: 0.15314025535110778
              precision    recall  f1-score   support

           0       0.13      0.18      0.15      3816
           1       0.15      0.18      0.16      3301
           2       0.13      0.12      0.12      3286
           3       0.10      0.36      0.16      3829
           4       0.20      0.17      0.18      3762
           5       0.14      0.08      0.10      1337
           6       0.24      0.15      0.19      2719
          

### KNN classifier with TfidfVectorizer

In [13]:
# Create classifier and use tf-idf vectors
KNeighborsClassifier_clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
print('*'*50)
print('KNeighbors classifier with TfidfVectorizer')
print(KNeighborsClassifier_clf)

# fit train data
KNeighborsClassifier_clf.fit(X_train_tfidf, y_train)

# predict the class and class proba.
predicted = KNeighborsClassifier_clf.predict(X_test_tfidf)
predicted_proba = KNeighborsClassifier_clf.predict_proba(X_test_tfidf)

show_result(predicted, predicted_proba, y_test)

**************************************************
KNeighbors classifier with TfidfVectorizer
KNeighborsClassifier(weights='distance')
**************************************************
predicted class of first 3 samples
[7 7 5]
**************************************************
predicted class proba. of first 3 samples
[[0.2  0.   0.2  0.   0.   0.   0.   0.4  0.   0.   0.2  0.   0.  ]
 [0.2  0.   0.2  0.   0.   0.   0.   0.4  0.   0.   0.2  0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.25 0.   0.25 0.25 0.   0.  ]]
Accuracy: 0.18679590687194894
              precision    recall  f1-score   support

           0       0.15      0.19      0.17      3816
           1       0.18      0.18      0.18      3301
           2       0.16      0.13      0.14      3286
           3       0.18      0.14      0.16      3829
           4       0.23      0.16      0.18      3762
           5       0.19      0.08      0.11      1337
           6       0.31      0.20      0.24      2719
          

### SVM classifier with CountVectorizer

In [15]:
# Create classifier and use count vectors
SVC_clf = SVC(probability=True)
print('*'*50)
print('SVM classifier with CountVectorizer')
print(SVC_clf)

# fit train data
SVC_clf.fit(X_train_counts, y_train)

# predict the class and class proba.
predicted = SVC_clf.predict(X_test_counts)
predicted_proba = SVC_clf.predict_proba(X_test_counts)

print('*'*50)
print('predicted class of first 3 test data')
print(predicted[:3])

print('*'*50)
print('predicted class proba. of first 3 test data')
print(predicted_proba[:3])

np.mean(predicted == y_test)
print('*'*50)
print('accuracy performance on test data')
print(np.mean(predicted == y_test))


**************************************************
SVM classifier with CountVectorizer
SVC(probability=True)


**************************************************
predicted class of first 3 test data
[1 1 7]
**************************************************
predicted class proba. of first 3 test data
[[0.12843748 0.15958017 0.0643388  0.08344182 0.07706299 0.015448
  0.02951556 0.13035405 0.09827882 0.06005393 0.08299881 0.0402352
  0.03025436]
 [0.12843748 0.15958017 0.0643388  0.08344182 0.07706299 0.015448
  0.02951556 0.13035405 0.09827882 0.06005393 0.08299881 0.0402352
  0.03025436]
 [0.06644106 0.06291574 0.04375748 0.06159269 0.10421445 0.07529481
  0.04019663 0.13714577 0.11420571 0.11745649 0.09056706 0.0366092
  0.04960289]]
**************************************************
accuracy performance on test data
0.2367161096507698


### SVM classifier with TfidfVectorizer

In [17]:
# Create classifier and use tf-idf vectors
SVC_clf = SVC(probability=True)
print('*'*50)
print('SVM classifier with TfidfVectorizer')
print(SVC_clf)

# fit train data
SVC_clf.fit(X_train_tfidf, y_train)

# predict the class and class proba.
predicted = SVC_clf.predict(X_test_tfidf)
predicted_proba = SVC_clf.predict_proba(X_test_tfidf)

print('*'*50)
print('predicted class of first 3 test data')
print(predicted[:3])

print('*'*50)
print('predicted class proba. of first 3 test data')
print(predicted_proba[:3])

np.mean(predicted == y_test)
print('*'*50)
print('accuracy performance on test data')
print(np.mean(predicted == y_test))


**************************************************
SVM classifier with TfidfVectorizer
SVC(probability=True)
**************************************************
predicted class of first 3 test data
[1 1 9]
**************************************************
predicted class proba. of first 3 test data
[[0.13641903 0.16833717 0.06238183 0.07113566 0.08615144 0.01600766
  0.0281073  0.12687915 0.10425791 0.05727538 0.07376283 0.0387223
  0.03056233]
 [0.13641903 0.16833717 0.06238183 0.07113566 0.08615144 0.01600766
  0.0281073  0.12687915 0.10425791 0.05727538 0.07376283 0.0387223
  0.03056233]
 [0.03883749 0.02989267 0.03940819 0.05955179 0.0880283  0.13078616
  0.06759664 0.11783641 0.08439914 0.15442623 0.08508593 0.04160121
  0.06254986]]
**************************************************
accuracy performance on test data
0.2455876830642133


## Save model

In [20]:
from joblib import dump
dump(KNeighborsClassifier_clf, 'KNeighborsClassifier_clf.joblib') 

['KNeighborsClassifier_clf.joblib']

## Predict

In [3]:
from joblib import load
KNeighborsClassifier_clf = load("KNeighborsClassifier_clf.joblib")

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
hw_test = pd.read_csv("ptt_data_test_63897.csv")
hw_test.fillna({'text':''}, inplace=True)
hw_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63897 entries, 0 to 63896
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  63897 non-null  int64  
 1   text        63897 non-null  object 
 2   label       0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


In [5]:
hw_test['title_SEG'] = [jieba.lcut(sent) for sent in hw_test['text']]
hw_test['title_SEG'] = hw_test['title_SEG'].apply(lambda x:' '.join(x))
hw_test.head(5)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/34/r649s2893p5576xp75qc7y9c0000gp/T/jieba.cache
Loading model cost 0.726 seconds.
Prefix dict has been built successfully.


,Unnamed: 0,text,label,title_SEG
0,0,"上次贈送版有人再送,我剛好拿到>///<",NaN,"上次 贈送 版 有人 再 送 , 我剛 好 拿到 > / / / <"
1,1,好亂呀 這部XD,NaN,好亂 呀 這部 XD
2,2,你覺得咧? 還有你覺得贏去年15%很多嗎?,NaN,你 覺得 咧 ? 還有 你 覺得贏 去年 15% 很多 嗎 ?
3,3,這時候不是「醫護先打，我不急」嗎？這是告訴,NaN,這時候 不是 「 醫護 先 打 ， 我 不 急 」 嗎 ？ 這是 告訴
4,4,tracert 66.151.54.140 xd,NaN,tracert 66.151 . 54.140 xd


In [6]:
# create feature vectors
tfidf_vect = TfidfVectorizer(max_features=1000)
hw_test_tfidf = tfidf_vect.fit_transform(hw_test['title_SEG'])

#prints the train data shape
print('train data shape using TfidfVectorizer')
print(hw_test_tfidf.shape)

train data shape using TfidfVectorizer
(63897, 1000)


In [7]:
# predict the class and class proba.
predicted = KNeighborsClassifier_clf.predict(hw_test_tfidf)

In [23]:
hw_submission = pd.read_csv("submisstion_63897.csv", index_col=[0])
hw_submission.head()

,label
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [24]:
hw_submission.label = predicted
hw_submission.head()

,label
0,0
1,7
2,2
3,11
4,8


In [25]:
hw_submission.to_csv("submission.csv")

In [27]:
pd.read_csv("submission.csv", index_col=[0])

,label
0,0
1,7
2,2
3,11
4,8
...,...
63892,0
63893,7
63894,0
63895,0
